In [1]:
import os
from aesim.simba import ProjectRepository, ThermalData
import matplotlib.pyplot as plt
from ipywidgets import Select, interactive
import numpy as np

In [2]:
igbt_xml_list = [filename for filename in os.listdir('./ThermalDataFile/') if filename.endswith('IGBT.xml')]

scrolling_menu = Select(options=igbt_xml_list, rows=len(igbt_xml_list))

In [3]:

def handle_selection(igbt_xml):
    file_path = os.path.join(os.getcwd(), "scrolling_menu_thermal_buck.jsimba")
    project = ProjectRepository(file_path)
    design = project.GetDesignByName('Design')
    design.TransientAnalysis.StopAtSteadyState = True
    igbt = design.Circuit.GetDeviceByName('IGBT1')
    for scope in igbt.Scopes:
        if scope.Name == 'Junction Temperature (°)' or scope.Name == 'Average Total Losses (W)':
            scope.Enabled = True

    igbt.ThermalData = ThermalData('ThermalDataFile/' + igbt_xml)

    job = design.TransientAnalysis.NewJob()
    status = job.Run()
    if str(status) != "OK": 
        print ("\nSimulation Failed ")
        print (job.Summary()[:-1])

    Tj = job.GetSignalByName('IGBT1 - Junction Temperature (°)').DataPoints
    Loss = job.GetSignalByName('IGBT1 - Average Total Losses (W)').DataPoints
    Tj_time = job.GetSignalByName('IGBT1 - Junction Temperature (°)').TimePoints
    Loss_time = job.GetSignalByName('IGBT1 - Average Total Losses (W)').TimePoints
        
    fig, ax = plt.subplots()
    ax.set_title("IGBT - Junction Temperature (°)")
    ax.set_ylabel('Junction Temperature (°)')
    ax.set_xlabel('time (s)')
    ax.plot(Tj_time,Tj)

    fig, ax = plt.subplots()
    ax.set_title("IGBT - Average Total Losses (W)")
    ax.set_ylabel('Average Total Losses (W)')
    ax.set_xlabel('time (s)')
    ax.plot(Loss_time,Loss)
    plt.show
    print("Selected item:", igbt_xml)

In [4]:

if os.environ.get('SIMBA_SCRIPT_TEST') == 'True':
    if igbt_xml_list:
        handle_selection(igbt_xml_list[0])
    else:
        print('No IGBT XML files found in ./ThermalDataFile/')
else:
    interactive_widget = interactive(handle_selection, igbt_xml=scrolling_menu)

    display(interactive_widget)

interactive(children=(Select(description='igbt_xml', options=('IGW15T120_IGBT.xml', 'IGW40T120_IGBT.xml', 'IGW…